In [1]:
import openai 
import tiktoken 
import chromadb 
import langchain

In [2]:
from langchain.custom_text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

In [3]:
#API key
import dotenv
dotenv.load_dotenv()

True

In [4]:
#Persist Directory
persist_directory = 'C:\ProjectLFG\PersistedEmbeddings'

In [5]:
# Now we can load the persisted database from disk, and use it as normal. 
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(disallowed_special=())
db = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

Setting Up Retriever

In [20]:
#Setting up Retriever
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 4}, # Number of relevant document to return
)

In [21]:
# Set up Chat
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k-0613") 

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [22]:
question = '''List Down all the tables that are beging used to create "DimUnifiedPartner" table'''
# To see relevant documents
docs = retriever.get_relevant_documents(question)

In [28]:
f = open("test-splits.sql", "w")

In [34]:
f.write(docs[0].page_content)
f.write( ''.join(' '.join(str(x) for x in docs[0].page_content)) )

5933

In [27]:
print(docs[0].page_content)

-- DBTITLE 1,Create DimUnifiedPartner_Final table 
DROP TABLE IF EXISTS DimUnifiedPartner_Final;
CREATE TABLE DimUnifiedPartner_Final AS 
SELECT  UnifiedPartnerKey
        ,SourceID AS PartnerID
        ,PartnerName
		,PartnerOrgType
		,MPNVOrgID AS VOrgID
		,DUP.PartnerGlobalID AS PartnerGlobalID
		,SourceName
		,PartnerOneSubID
		,DUP.PartnerOneSubKey
		,DUP.PartnerOneID
		,CASE WHEN DUP.PartnerOneID = -9999 THEN 'Unmatched Partners' ELSE PartnerOneName END AS PartnerOneName
		,IsLSP
		,Area 
		,Region 
		,SubRegion
		,Subsidiary
        ,SubsidiaryID
        ,CASE WHEN UnifiedPartnerKey = 2926514 THEN 'United States'
        WHEN DSR.ResellerTPID IS NOT NULL THEN DSR.VOrgArea ELSE  DUP.VOrgArea END AS VOrgArea
        ,CASE WHEN UnifiedPartnerKey = 2926514 THEN 'United States'
        WHEN DSR.ResellerTPID IS NOT NULL THEN DSR.PartnerGlobalArea ELSE  DUP.PartnerGlobalArea END AS PartnerGlobalArea
		,CASE WHEN DSR.ResellerTPID IS NOT NULL THEN DSR.VOrgRegion ELSE DUP.VOrgRegion END A

In [24]:
#Result
result = qa(question)
print(result['answer'])

The "DimUnifiedPartner" table is created using the following tables:
1. DimUnifiedPartner_tmp
2. PDM_tmp
3. MapOpptyPartner_tmp
